<img src="https://github.com/fjmeyer/HydroSAR/raw/master/HydroSARbanner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Load HyP3 Data for Flood Depth Mapping</b><img style="padding: 7px" src="UAFLogo_A_647.png" width="170" align="right"/></font>

 <font size="4"> <b> Part of NASA A.37 Project:</b> Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters   <br>
<font size="4"> <b> PI:</b>Franz J. Meyer <br>

<br>
<font size="4"> <b> Batuhan Osmanoglu, MinJeong Jo; NASA Goddard Space Fligth Center </b> <br>
</font>

<font size="3"> This notebook uses flood-extent data from an ASF-HyP3 subscription. DEM tiles are extracted and mosaicked to a consistent large-scale product for use in other HydroSAR workflows. Either the production DEM can be used to generate HAND or alternative HAND DEM can be downloaded. Flood depth can be estimated using HAND.  
    
<b><u>Complete Flood Depth Mapping Sequence:</u></b>

This notebook is the first step in the <b>Flood Depth Mapping Sequence</b>, which consists of running the following notebooks in the order given:
    
1. **Complete Lab 2** [Lab2-SurfaceWaterExtentMapping.ipynb](./Lab2-SurfaceWaterExtentMapping.ipynb) 
2. **Prepare HAND layer:**
   - Use existing HAND DEM: Bangladesh_Training_DEM_hand.tif (Skip step 2)
   - OR Generate HAND using DEM by running [Big_Hand_notebook.ipynb](./Big_Hand_notebook.ipynb)
   - OR Download HAND using [Download_HAND_from_GEE.ipynb](./Download_HAND_from_GEE.ipynb)
3. **Flood Depth Estimation:** Run [Flood_Depth_Estimation_with_Flood_Extent_Maps.ipynb](./Flood_Depth_Estimation_with_Flood_Extent_Maps.ipynb) 
</font></font>

<hr>
<font face="Calibri" size="5" color="darkred"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


# Importing Relevant Python Packages

<font face="Calibri">
<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
</font>
<br>
<font face="Calibri" size="3"><b>Our first step is to import them:</b> </font>

In [1]:
%%capture
import copy
import os
import glob
import json # for loads
import html
import shutil
import re
import gdal
import numpy as np
from IPython.display import HTML, display, clear_output, Markdown
from ipywidgets import interact, Button, HBox, Layout

try:
    from tqdm.auto import tqdm
except:
    !pip install tqdm
    from tqdm.auto import tqdm
    
from asf_notebook import EarthdataLogin
from asf_notebook import new_directory
from asf_notebook import get_wget_cmd
from asf_notebook import asf_unzip
from asf_notebook import path_exists
from asf_notebook import get_RTC_polarizations
from asf_notebook import get_hyp3_subscriptions
from asf_notebook import gui_date_picker
from asf_notebook import get_subscription_products_info
from asf_notebook import get_subscription_granule_names_ids
from asf_notebook import date_from_product_name
from asf_notebook import get_products_dates_insar
from asf_notebook import get_product_info
from asf_notebook import select_parameter
from asf_notebook import select_mult_parameters
from asf_notebook import get_slider_vals
from asf_notebook import input_path
from asf_notebook import handle_old_data
from asf_notebook import get_power_set

# Select the folder you have the Flood Extent Map

<font face="Calibri">
<font size="3"> This notebook assumes that you've a flood extent map available in your Notebook environment. We start with selecting the folder where you have the data. 
</font>

<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. Setup an EarthData account if you do not yet have one. <font color='rgba(200,0,0,0.2)'><b>Note that EarthData's End User License Agreement (EULA) applies when accessing the Hyp3 API from this notebook. If you have not acknowleged the EULA in EarthData, you will need to navigate to <a href="https://earthdata.nasa.gov/" target="_blank">EarthData's home page</a> and complete that process.</b></font>
<br><br>
<b>Download data if needed</b> </font> 

<hr>
<font face="Calibri" size="3"> Before we download anything, create a working directory for this analysis and change into it. 
<br><br>
<b>Select or create a working directory for the analysis:</b></font>

# Version History

<font face="Calibri" size="2"> <i>LoadHyP3Data-FullFrame.ipynb - Version 1.0 - December 2020 
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>First Version of this notebook</li> 
    </ul>
    </i>
</font>